In [ ]:
# hide

%load_ext nb_black
%config InlineBackend.figure_format="retina"

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients

# from will_it_saturate.gevent_client import GeventClient
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

In [ ]:
def get_results(name="intel_wrk_results.db"):
    repository = SqliteRepository.build_repository(name)
    register_default_tables(repository)
    results = repository.tables["result"]
    return results

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

# control_host = "localhost"
control_host = "staging.wersdoerfer.de"
control_port = 8100

client_control_host = Host(name=control_host, port=control_port)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_clients = [
    clients.WrkClient(name="wrk"),
    # clients.AioHttpClient(name="aiohttp"),
    # clients.HttpxClient(name="httpx"),
]
# benchmark_client = clients.WrkClient(name="wrk")

server_control_host = Host(name=control_host, port=control_port)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server_classes = [
    servers.NginxFileServer,
    servers.FastAPIUvicornServer,
    servers.NginxSendfileServer,
    servers.CaddyFileServer,
]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte, file_size, duration = 8, 10**5, 10
for server_class in server_classes:
    server = server_control_client.get_or_create_server(
        server_class(host="localhost", port=5001)
    )
    epoch = Epoch(file_size=file_size, duration=duration)
    files = server_control_client.get_or_create_files(epoch)
    epoch.urls = [server.file_to_url(file) for file in files]
    for benchmark_client in benchmark_clients:
        result = Result(
            server=server,
            client=benchmark_client,
            server_details=server_details,
            client_details=client_details,
            file_size=epoch.file_size,
            complete_size=epoch.complete_size,
        )
        result.elapsed = client_control_client.measure(benchmark_client, epoch)
        results = get_results()
        result_id = results.add_result(result)
    server_control_client.stop_server(server)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results = get_results()
results_from_database = results.get_results()

show_columns = [
    "server",
    "client",
    "elapsed",
    "file_size_h",
    "bytes_per_second_h",
    "arch",
]

cols = set(show_columns + ["bytes_per_second"])

df = pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)
df.sort_values(["file_size_h", "bytes_per_second"], ascending=False)[show_columns]

,server,client,elapsed,file_size_h,bytes_per_second_h,arch
0,nginx/minimal,wrk,0.695651,97.66KB,1.67GB,x86_64
2,nginx/sendfile,wrk,0.718180,97.66KB,1.62GB,x86_64
3,caddy,wrk,0.880563,97.66KB,1.32GB,x86_64
1,fastAPI/uvicorn,wrk,6.153709,97.66KB,193.72MB,x86_64


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

df.sort_values(["file_size_h", "bytes_per_second"], ascending=False)[
    show_columns
].to_html(index=False, col_space=100)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th style="min-width: 100px;">server</th>\n      <th style="min-width: 100px;">client</th>\n      <th style="min-width: 100px;">elapsed</th>\n      <th style="min-width: 100px;">file_size_h</th>\n      <th style="min-width: 100px;">bytes_per_second_h</th>\n      <th style="min-width: 100px;">arch</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>nginx/minimal</td>\n      <td>wrk</td>\n      <td>0.695651</td>\n      <td>97.66KB</td>\n      <td>1.67GB</td>\n      <td>x86_64</td>\n    </tr>\n    <tr>\n      <td>nginx/sendfile</td>\n      <td>wrk</td>\n      <td>0.718180</td>\n      <td>97.66KB</td>\n      <td>1.62GB</td>\n      <td>x86_64</td>\n    </tr>\n    <tr>\n      <td>caddy</td>\n      <td>wrk</td>\n      <td>0.880563</td>\n      <td>97.66KB</td>\n      <td>1.32GB</td>\n      <td>x86_64</td>\n    </tr>\n    <tr>\n      <td>fastAPI/uvicorn</td>\n      <td>wrk</td>\n      <td>6.1537

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results = get_results()
results_from_database = results.get_results()

show_columns = [
    "server",
    "client",
    "elapsed",
    "file_size_h",
    "bytes_per_second_h",
    "arch",
]

cols = set(show_columns + ["bytes_per_second"])

df = pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)
df.sort_values(["file_size_h", "bytes_per_second"], ascending=False)[show_columns]

,server,client,elapsed,file_size_h,bytes_per_second_h,arch
2,nginx/sendfile,wrk,0.713600,97.66KB,1.63GB,x86_64
3,caddy,wrk,0.888603,97.66KB,1.31GB,x86_64
0,nginx/minimal,wrk,0.899590,97.66KB,1.29GB,x86_64
1,fastAPI/uvicorn,wrk,11.827921,97.66KB,100.79MB,x86_64


In [ ]:
# dont_test

df.sort_values(["file_size_h", "bytes_per_second"], ascending=False)[show_columns].to_html(index=False, col_space=100)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th style="min-width: 100px;">server</th>\n      <th style="min-width: 100px;">client</th>\n      <th style="min-width: 100px;">elapsed</th>\n      <th style="min-width: 100px;">file_size_h</th>\n      <th style="min-width: 100px;">bytes_per_second_h</th>\n      <th style="min-width: 100px;">arch</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>nginx/sendfile</td>\n      <td>wrk</td>\n      <td>0.252434</td>\n      <td>97.66KB</td>\n      <td>4.61GB</td>\n      <td>arm64</td>\n    </tr>\n    <tr>\n      <td>nginx/minimal</td>\n      <td>wrk</td>\n      <td>0.287506</td>\n      <td>97.66KB</td>\n      <td>4.05GB</td>\n      <td>arm64</td>\n    </tr>\n    <tr>\n      <td>caddy</td>\n      <td>wrk</td>\n      <td>0.481639</td>\n      <td>97.66KB</td>\n      <td>2.42GB</td>\n      <td>arm64</td>\n    </tr>\n    <tr>\n      <td>fastAPI/uvicorn</td>\n      <td>wrk</td>\n      <td>4.228977<

In [ ]:
# dont_test

df.style.to_html()

'<style type="text/css">\n</style>\n<table id="T_938ee">\n  <thead>\n    <tr>\n      <th class="blank level0" >&nbsp;</th>\n      <th id="T_938ee_level0_col0" class="col_heading level0 col0" >server</th>\n      <th id="T_938ee_level0_col1" class="col_heading level0 col1" >client</th>\n      <th id="T_938ee_level0_col2" class="col_heading level0 col2" >elapsed</th>\n      <th id="T_938ee_level0_col3" class="col_heading level0 col3" >file_size_h</th>\n      <th id="T_938ee_level0_col4" class="col_heading level0 col4" >bytes_per_second</th>\n      <th id="T_938ee_level0_col5" class="col_heading level0 col5" >bytes_per_second_h</th>\n      <th id="T_938ee_level0_col6" class="col_heading level0 col6" >arch</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th id="T_938ee_level0_row0" class="row_heading level0 row0" >0</th>\n      <td id="T_938ee_row0_col0" class="data row0 col0" >nginx/minimal</td>\n      <td id="T_938ee_row0_col1" class="data row0 col1" >wrk</td>\n      <td id="T_938